<a href="https://colab.research.google.com/github/KetakiN/Springboard/blob/main/NYSDOH_Ket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
chunk_size=50000
df1=pd.DataFrame()
for chunk in pd.read_csv("/content/sample_data/nysdohdatafile.csv",chunksize=chunk_size,low_memory=False):
    df2=chunk.sample(frac=0.05)
    df=pd.concat([df1, df2])
    df1=df

In [19]:
df.shape

(31684, 35)

In [21]:
df.head(2)

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,Discharge Year,CCS Diagnosis Code,CCS Diagnosis Description,CCS Procedure Code,CCS Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs,Ratio of Total Costs to Total Charges
30714,Western NY,Erie,1401014.0,207.0,Buffalo General Medical Center,30 to 49,142,F,White,Not Span/Hispanic,7,Emergency,Home w/ Home Health Services,2014.0,131.0,Respiratory failure; insufficiency; arrest (ad...,0.0,NO PROC,133.0,Respiratory Failure,4.0,Diseases and Disorders of the Respiratory System,3.0,Major,Major,Medical,Medicare,Self-Pay,Self-Pay,0.0,N,Y,"32,006.13","13,981.32",0.436833
22745,Western NY,Erie,1401014.0,207.0,Buffalo General Medical Center,50 to 69,NaN,M,Black/African American,Not Span/Hispanic,27,Emergency,Home w/ Home Health Services,2014.0,5.0,HIV infection,216.0,RESP INTUB/MECH VENTIL,890.0,HIV w multiple major HIV related conditions,24.0,Human Immunodeficiency Virus Infections,4.0,Extreme,Extreme,Medical,Medicaid,Self-Pay,NaN,0.0,N,Y,"166,734.73","66,807.42",0.400681


In [23]:
df.dtypes

Hospital Service Area                     object
Hospital County                           object
Operating Certificate Number             float64
Permanent Facility Id                    float64
Facility Name                             object
Age Group                                 object
Zip Code - 3 digits                       object
Gender                                    object
Race                                      object
Ethnicity                                 object
Length of Stay                            object
Type of Admission                         object
Patient Disposition                       object
Discharge Year                           float64
CCS Diagnosis Code                       float64
CCS Diagnosis Description                 object
CCS Procedure Code                       float64
CCS Procedure Description                 object
APR DRG Code                             float64
APR DRG Description                       object
APR MDC Code        

In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
df.isnull().sum()

Hospital Service Area                       72
Hospital County                             72
Operating Certificate Number                72
Permanent Facility Id                       72
Facility Name                                0
Age Group                                    0
Zip Code - 3 digits                        426
Gender                                       0
Race                                         0
Ethnicity                                    0
Length of Stay                               0
Type of Admission                            0
Patient Disposition                          0
Discharge Year                               0
CCS Diagnosis Code                           0
CCS Diagnosis Description                    0
CCS Procedure Code                           0
CCS Procedure Description                    0
APR DRG Code                                 0
APR DRG Description                          0
APR MDC Code                                 0
APR MDC Descr

In [26]:
# replace 120+ length of stay with 120, which will be considered the maximum length of stay category
df.loc[df['Length of Stay']=='120 +','Length of Stay']='120'

In [27]:
df['Length of Stay']=df['Length of Stay'].astype(float)

Looking at missing values, zip code Zip Code - 3 digits Type is Char. Length is 3. The first three digits of the patient's
zip code. Blank for:- population size less than 20,000- abortion records, or- cell size less than 10 on population,classification strata. 
“OOS” are Out of State zipcodes

In [28]:
# the missing nan zip codes will be replaced with string '0'
df['Zip Code - 3 digits']=df['Zip Code - 3 digits'].fillna(0).astype(str)

Change column names to replace spaces with undersore, keep relevant columns and drop columns that are not helpful in predicting length of stay: Payment typology 2 and 3 ( secondery and tertairy insurances, APR codes, birth weight columns( as majority of our patients are adults, abortion edit indicator,a s all abortion row information has been removed, charges and costs)

In [29]:
df.columns=df.columns.str.replace(' ','_')
df.columns=df.columns.str.replace('-','_')
df.columns

Index(['Hospital_Service_Area', 'Hospital_County',
       'Operating_Certificate_Number', 'Permanent_Facility_Id',
       'Facility_Name', 'Age_Group', 'Zip_Code___3_digits', 'Gender', 'Race',
       'Ethnicity', 'Length_of_Stay', 'Type_of_Admission',
       'Patient_Disposition', 'Discharge_Year', 'CCS_Diagnosis_Code',
       'CCS_Diagnosis_Description', 'CCS_Procedure_Code',
       'CCS_Procedure_Description', 'APR_DRG_Code', 'APR_DRG_Description',
       'APR_MDC_Code', 'APR_MDC_Description', 'APR_Severity_of_Illness_Code',
       'APR_Severity_of_Illness_Description', 'APR_Risk_of_Mortality',
       'APR_Medical_Surgical_Description', 'Payment_Typology_1',
       'Payment_Typology_2', 'Payment_Typology_3', 'Birth_Weight',
       'Abortion_Edit_Indicator', 'Emergency_Department_Indicator',
       'Total_Charges', 'Total_Costs',
       'Ratio_of_Total_Costs_to_Total_Charges'],
      dtype='object')

In [30]:
df=df.drop(['Operating_Certificate_Number','Permanent_Facility_Id','Discharge_Year','Patient_Disposition','APR_DRG_Code', 'APR_DRG_Description',
       'APR_MDC_Code', 'APR_MDC_Description', 'APR_Severity_of_Illness_Code',
       'APR_Severity_of_Illness_Description', 'APR_Risk_of_Mortality',
       'APR_Medical_Surgical_Description', 'Payment_Typology_2', 'Payment_Typology_3', 'Birth_Weight',
       'Abortion_Edit_Indicator',
       'Total_Charges', 'Total_Costs',
       'Ratio_of_Total_Costs_to_Total_Charges'],axis=1)

In [31]:
df.columns

Index(['Hospital_Service_Area', 'Hospital_County', 'Facility_Name',
       'Age_Group', 'Zip_Code___3_digits', 'Gender', 'Race', 'Ethnicity',
       'Length_of_Stay', 'Type_of_Admission', 'CCS_Diagnosis_Code',
       'CCS_Diagnosis_Description', 'CCS_Procedure_Code',
       'CCS_Procedure_Description', 'Payment_Typology_1',
       'Emergency_Department_Indicator'],
      dtype='object')

In [32]:
df=df.dropna(axis=0,how='any')

In [37]:
df=df.drop('Ethnicity',axis=1)

In [36]:
df.Race.unique()

array(['White', 'Black/African American', 'Other Race', 'Multi-racial'],
      dtype=object)

In [40]:
df.isnull().sum()

Hospital_Service_Area             0
Hospital_County                   0
Facility_Name                     0
Age_Group                         0
Zip_Code___3_digits               0
Gender                            0
Race                              0
Length_of_Stay                    0
Type_of_Admission                 0
CCS_Diagnosis_Code                0
CCS_Diagnosis_Description         0
CCS_Procedure_Code                0
CCS_Procedure_Description         0
Payment_Typology_1                0
Emergency_Department_Indicator    0
dtype: int64

In [41]:
df.dtypes

Hospital_Service_Area              object
Hospital_County                    object
Facility_Name                      object
Age_Group                          object
Zip_Code___3_digits                object
Gender                             object
Race                               object
Length_of_Stay                    float64
Type_of_Admission                  object
CCS_Diagnosis_Code                float64
CCS_Diagnosis_Description          object
CCS_Procedure_Code                float64
CCS_Procedure_Description          object
Payment_Typology_1                 object
Emergency_Department_Indicator     object
dtype: object

In [42]:

#from google.colab import files

In [43]:
#df.to_csv('nysdoh_cleaned.csv')

In [44]:
#files.download('nysdoh_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
df['CCS_Diagnosis_Description'].describe()

count        31612
unique         251
top       Liveborn
freq          2817
Name: CCS_Diagnosis_Description, dtype: object

International Classification of Diseases, Clinical Modification (ICD-9-CM) is an adaption created by the U.S. National Center for Health Statistics (NCHS) and used in assigning diagnostic and procedure codes associated with inpatient, outpatient, and physician office utilization in the United States.There are 17 main categories:
https://en.wikipedia.org/wiki/List_of_ICD-9_codes
